In [1]:
from pyspark.sql import Window
from pyspark.sql.functions import explode
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf, size
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs, date_format, \
    from_utc_timestamp, expr, min, max

In [2]:
df = spark.read.json('/media/backup/datasets/mercado_livre/train_dataset.jl')
df.show(2)

+-----------+--------------------+
|item_bought|        user_history|
+-----------+--------------------+
|    1748830|[[1786148, 2019-1...|
|     228737|[[643652, 2019-10...|
+-----------+--------------------+
only showing top 2 rows



In [3]:
df_meta = spark.read.json('/media/backup/datasets/mercado_livre/item_data.jl')
df_meta.show(2)

+-----------+---------+--------------------+-------+----------+----------+--------------------+
|category_id|condition|           domain_id|item_id|     price|product_id|               title|
+-----------+---------+--------------------+-------+----------+----------+--------------------+
|  MLM170527|      new|MLM-INDIVIDUAL_HO...| 111260|1150000.00|      null|Casa Sola En Vent...|
|  MLM151595|      new|     MLM-VIDEO_GAMES| 871377|   1392.83|  15270800|Resident Evil Ori...|
+-----------+---------+--------------------+-------+----------+----------+--------------------+
only showing top 2 rows



In [4]:
df.take(1)

[Row(item_bought=1748830, user_history=[Row(event_info='1786148', event_timestamp='2019-10-19T11:25:42.444-0400', event_type='view'), Row(event_info='1786148', event_timestamp='2019-10-19T11:25:57.487-0400', event_type='view'), Row(event_info='RELOGIO SMARTWATCH', event_timestamp='2019-10-19T11:26:07.063-0400', event_type='search'), Row(event_info='1615991', event_timestamp='2019-10-19T11:27:26.879-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T11:28:36.558-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T11:28:40.827-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T11:30:42.089-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T21:51:29.622-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T21:52:09.281-0400', event_type='view'), Row(event_info='1615991', event_timestamp='2019-10-19T21:52:41.863-0400', event_type='view'), Row(eve

In [5]:
df = df.withColumn("session_id", F.monotonically_increasing_id())

In [6]:
df.show()

+-----------+--------------------+----------+
|item_bought|        user_history|session_id|
+-----------+--------------------+----------+
|    1748830|[[1786148, 2019-1...|         0|
|     228737|[[643652, 2019-10...|         1|
|    1909110|[[248595, 2019-10...|         2|
|    1197370|[[RADIOBOSS, 2019...|         3|
|    2049207|[[AMAZFIT BIP, 20...|         4|
|    1046119|[[TAMPA TRASEIRA ...|         5|
|    1206282|[[400631, 2019-10...|         6|
|    1393318|[[SECADOR TAIFF, ...|         7|
|    2056361|[[AUDIFONOS BASS,...|         8|
|    1916890|[[1142550, 2019-1...|         9|
|     392483|[[HARMAN KARDON, ...|        10|
|     107670|[[2083291, 2019-1...|        11|
|     537473|[[ORGANIZADOR SAP...|        12|
|     673499|[[307861, 2019-09...|        13|
|      65317|[[773066, 2019-10...|        14|
|    1371364|[[FILHOTE CACHORR...|        15|
|    1877560|[[876310, 2019-09...|        16|
|        232|[[JBL GO 2, 2019-...|        17|
|    1228971|[[REMEDIOS EMAGRE...|

In [7]:
df = df.withColumn("event", explode(df.user_history))

df = df.withColumn('event_info', col("event").getItem("event_info"))\
        .withColumn('event_timestamp', col("event").getItem("event_timestamp"))\
        .withColumn('event_type', col("event").getItem("event_type"))

df.show()

+-----------+--------------------+----------+--------------------+------------------+--------------------+----------+
|item_bought|        user_history|session_id|               event|        event_info|     event_timestamp|event_type|
+-----------+--------------------+----------+--------------------+------------------+--------------------+----------+
|    1748830|[[1786148, 2019-1...|         0|[1786148, 2019-10...|           1786148|2019-10-19T11:25:...|      view|
|    1748830|[[1786148, 2019-1...|         0|[1786148, 2019-10...|           1786148|2019-10-19T11:25:...|      view|
|    1748830|[[1786148, 2019-1...|         0|[RELOGIO SMARTWAT...|RELOGIO SMARTWATCH|2019-10-19T11:26:...|    search|
|    1748830|[[1786148, 2019-1...|         0|[1615991, 2019-10...|           1615991|2019-10-19T11:27:...|      view|
|    1748830|[[1786148, 2019-1...|         0|[1615991, 2019-10...|           1615991|2019-10-19T11:28:...|      view|
|    1748830|[[1786148, 2019-1...|         0|[1615991, 2

In [8]:
_df = df.select("session_id", "event_timestamp", "event_info", "event_type")
_df.show()

+----------+--------------------+------------------+----------+
|session_id|     event_timestamp|        event_info|event_type|
+----------+--------------------+------------------+----------+
|         0|2019-10-19T11:25:...|           1786148|      view|
|         0|2019-10-19T11:25:...|           1786148|      view|
|         0|2019-10-19T11:26:...|RELOGIO SMARTWATCH|    search|
|         0|2019-10-19T11:27:...|           1615991|      view|
|         0|2019-10-19T11:28:...|           1615991|      view|
|         0|2019-10-19T11:28:...|           1615991|      view|
|         0|2019-10-19T11:30:...|           1615991|      view|
|         0|2019-10-19T21:51:...|           1615991|      view|
|         0|2019-10-19T21:52:...|           1615991|      view|
|         0|2019-10-19T21:52:...|           1615991|      view|
|         0|2019-10-19T21:54:...|           1615991|      view|
|         0|2019-10-19T21:54:...|           1615991|      view|
|         0|2019-10-19T21:54:...|       

In [9]:
from pyspark.sql import Window
import pyspark.sql.functions as psf

_df2 = df.groupBy("session_id").agg(max(df.event_timestamp).alias("event_timestamp"), 
                                    max(df.item_bought).alias("event_info"))
_df2 = _df2.withColumn('event_type', lit("buy"))
_df2 = _df2.withColumn('event_timestamp', F.date_add(_df2['event_timestamp'], 1))

_df2.show()

+----------+---------------+----------+----------+
|session_id|event_timestamp|event_info|event_type|
+----------+---------------+----------+----------+
|        26|     2019-10-26|   1567189|       buy|
|        29|     2019-10-18|   1335130|       buy|
|       474|     2019-10-10|   1495122|       buy|
|       964|     2019-10-09|    892343|       buy|
|      1677|     2019-10-24|    815397|       buy|
|      1697|     2019-10-26|   1846850|       buy|
|      1806|     2019-10-28|    223369|       buy|
|      1950|     2019-09-28|    213136|       buy|
|      2040|     2019-10-04|    394243|       buy|
|      2214|     2019-10-12|    673207|       buy|
|      2250|     2019-10-23|   1554497|       buy|
|      2453|     2019-10-08|   1880554|       buy|
|      2509|     2019-10-30|    985132|       buy|
|      2529|     2019-10-05|   1867053|       buy|
|      2927|     2019-10-08|    119967|       buy|
|      3091|     2019-10-25|   1476214|       buy|
|      3506|     2019-10-19|   

In [10]:
dataset = _df.union(_df2)
dataset.show(2)


+----------+--------------------+----------+----------+
|session_id|     event_timestamp|event_info|event_type|
+----------+--------------------+----------+----------+
|         0|2019-10-19T11:25:...|   1786148|      view|
|         0|2019-10-19T11:25:...|   1786148|      view|
+----------+--------------------+----------+----------+
only showing top 2 rows



In [14]:
dataset.filter(dataset.session_id == 4310).orderBy(col("event_timestamp")).show(1000)

+----------+--------------------+--------------------+----------+
|session_id|     event_timestamp|          event_info|event_type|
+----------+--------------------+--------------------+----------+
|      4310|2019-10-25T07:19:...|  PROGRESSIVA FORMOL|    search|
|      4310|2019-10-25T07:19:...|         PROGRESSIVA|    search|
|      4310|2019-10-25T07:20:...|              617673|      view|
|      4310|2019-10-25T07:20:...|         PROGRESSIVA|    search|
|      4310|2019-10-25T07:20:...|         PROGRESSIVA|    search|
|      4310|2019-10-25T07:20:...|         PROGRESSIVA|    search|
|      4310|2019-10-25T07:20:...|  PROGRESSIVA FORMOL|    search|
|      4310|2019-10-25T09:00:...|   MARIA ESCANDALOSA|    search|
|      4310|2019-10-25T09:00:...|              756696|      view|
|      4310|2019-10-25T09:00:...|   MARIA ESCANDALOSA|    search|
|      4310|2019-10-25T09:00:...|              397019|      view|
|      4310|2019-10-25T09:00:...|   MARIA ESCANDALOSA|    search|
|      431

In [12]:
dataset.filter(dataset.event_type != "search").show()

+----------+--------------------+----------+----------+
|session_id|     event_timestamp|event_info|event_type|
+----------+--------------------+----------+----------+
|         0|2019-10-19T11:25:...|   1786148|      view|
|         0|2019-10-19T11:25:...|   1786148|      view|
|         0|2019-10-19T11:27:...|   1615991|      view|
|         0|2019-10-19T11:28:...|   1615991|      view|
|         0|2019-10-19T11:28:...|   1615991|      view|
|         0|2019-10-19T11:30:...|   1615991|      view|
|         0|2019-10-19T21:51:...|   1615991|      view|
|         0|2019-10-19T21:52:...|   1615991|      view|
|         0|2019-10-19T21:52:...|   1615991|      view|
|         0|2019-10-19T21:54:...|   1615991|      view|
|         0|2019-10-19T21:54:...|   1615991|      view|
|         0|2019-10-19T21:54:...|   1615991|      view|
|         0|2019-10-19T22:00:...|   1615991|      view|
|         0|2019-10-20T10:36:...|   1615991|      view|
|         0|2019-10-20T10:37:...|   1615991|    

In [2]:
from datetime import datetime, timedelta


In [15]:
from pyspark.sql.functions import col, udf, size

df = spark.read.option("delimiter", ",").csv("/media/workspace/triplet_session/output/mercado_livre/dataset/session_dataset.csv", header=True, inferSchema=True)
df = df.withColumnRenamed("session_id", "SessionID")\
    .withColumnRenamed("event_timestamp2", "Timestamp")\
    .withColumnRenamed("event_info", "ItemID")\
    .withColumn("Timestamp", col("Timestamp").cast("timestamp"))\
    .orderBy(col('Timestamp'), col('SessionID')).select("SessionID", "ItemID", "Timestamp", "event_type")

# Remove Search event

In [16]:
df.count()

12412331

In [17]:
df = df.filter(df.event_type != "search")

In [18]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

df2 = df.limit(10000).cache()

df2.show(2)

+-----------+------+--------------------+----------+
|  SessionID|ItemID|           Timestamp|event_type|
+-----------+------+--------------------+----------+
|51539634491|873326|2019-09-24 08:42:...|      view|
|77309438469|527037|2019-09-24 08:43:...|      view|
+-----------+------+--------------------+----------+
only showing top 2 rows



In [1]:
df2.schedule()

NameError: name 'df2' is not defined

In [7]:
#2019-10-03T00:00:00.462-0400
df2.withColumn("Timestamp2", from_unixtime(unix_timestamp('Timestamp', 'yyyy-MM-dd HH:mm:ss.sssz'))).show(2)

+---------+-------+---------+----------+----------+
|SessionID| ItemID|Timestamp|event_type|Timestamp2|
+---------+-------+---------+----------+----------+
|        0|1786148|     null|      view|      null|
|        0|1786148|     null|      view|      null|
+---------+-------+---------+----------+----------+
only showing top 2 rows



In [8]:
from dateutil.parser import parse
from pyspark.sql.types import TimestampType


parse("2019-10-03T00:00:00.462-0400")

datetime.datetime(2019, 10, 3, 0, 0, 0, 462000, tzinfo=tzoffset(None, -14400))

In [9]:
parse_date =  udf (lambda x: parse(x), TimestampType())

df2.withColumn('Timestamp2', parse_date(col('Timestamp'))).show(2)

Py4JJavaError: An error occurred while calling o85.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 244, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.3.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/opt/spark-2.3.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-2.3.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 331, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark-2.3.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 140, in dump_stream
    for obj in iterator:
  File "/opt/spark-2.3.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspa

In [9]:
df = df.dropDuplicates(['SessionID', 'ItemID', 'event_type'])

In [15]:
max_timestamp = df.select(max(col('Timestamp'))).collect()[0]['max(Timestamp)']
init_timestamp = max_timestamp - timedelta(days = 8)

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'

In [ ]:
init_timestamp, max_timestamp